In [ ]:
# # tree евристика
# if __name__ == "__main__":
#     import numpy as np
#     import pandas as pd
#     from tqdm import tqdm
#     from river import tree
#     import random

#     # Фиксация глобальных сидов
#     GLOBAL_SEED = 42
#     np.random.seed(GLOBAL_SEED)
#     random.seed(GLOBAL_SEED)

#     class IsingEnvironment:
#         def __init__(self, J, seed=None):
#             self.J = J
#             self.N = J.shape[0]
#             self.rng = np.random.RandomState(seed)
#             self.state = self.rng.choice([-1, 1], size=self.N)
#             self.no_improvement_streak = 0
            
#             # Инициализация лучшего состояния и энергии
#             self.best_energy = self.calculate_energy(self.state)
#             self.best_state = self.state.copy()
#             # Текущая энергия для инкрементальных обновлений
#             self.energy = self.best_energy
        
#         def calculate_energy(self, spins):
#             return -np.sum(np.triu(self.J * np.outer(spins, spins)))
        
#         def delta_energy_if_flip(self, i):
#             # ΔE = 2 * s_i * sum_j J_ij s_j
#             return 2.0 * self.state[i] * np.dot(self.J[i], self.state)
        
#         def flip_spin(self, i):
#             # Быстрый инкрементальный пересчет энергии
#             delta_energy = self.delta_energy_if_flip(i)
#             # Награда: нормированный минус ΔE (лучше, когда ΔE отрицательно)
#             max_delta = 2.0 * np.sum(np.abs(self.J[i])) + 1e-9
#             reward = (-delta_energy) / max_delta
            
#             # Штраф за отсутствие улучшений
#             if delta_energy >= 0:
#                 self.no_improvement_streak += 1
#                 reward -= 0.01 * self.no_improvement_streak
#             else:
#                 self.no_improvement_streak = 0
            
#             # Применяем переворот и обновляем энергию
#             self.state[i] *= -1
#             self.energy += delta_energy
            
#             # Обновление лучшего состояния
#             if self.energy < self.best_energy:
#                 self.best_energy = self.energy
#                 self.best_state = self.state.copy()
                
#             return reward
        
#         def get_state(self):
#             return tuple(self.state)
        
#         def get_best_state(self):
#             return self.best_state.copy()
        
#         def greedy_polish(self, max_steps=None):
#             # Жадная локальная оптимизация по наилучшему улучшению
#             steps_budget = max_steps if max_steps is not None else 2 * self.N
#             for _ in range(steps_budget):
#                 interaction = self.J @ self.state
#                 deltas = 2.0 * self.state * interaction
#                 i_best = int(np.argmin(deltas))
#                 if deltas[i_best] < 0:
#                     # Применить лучший переворот
#                     self.state[i_best] *= -1
#                     self.energy += deltas[i_best]
#                     if self.energy < self.best_energy:
#                         self.best_energy = self.energy
#                         self.best_state = self.state.copy()
#                 else:
#                     break

#     class TreeQAgent:
#         def __init__(self, env, gamma=0.95, 
#                     initial_temp=1.0, final_temp=0.01, 
#                     temp_decay_rate=0.9995, seed=None):
#             self.env = env
#             self.gamma = gamma
#             self.temperature = initial_temp
#             self.final_temp = final_temp
#             self.temp_decay_rate = temp_decay_rate
#             self.N = env.N
            
#             self.rng = np.random.RandomState(seed)
#             # Упрощенный скейлер (только смещение)
#             self.mean = np.zeros(self.N)
#             self.n_seen = 0
            
#             # Инициализация моделей Hoeffding Tree
#             self.q_models = [
#                 tree.HoeffdingTreeRegressor(
#                     grace_period=100,
#                     max_depth=10
#                 ) for i in range(self.N)
#             ]
        
#         def _state_to_dict(self, state):
#             """Преобразует вектор состояния в словарь признаков."""
#             state = np.array(state)
#             state_shifted = state - self.mean
#             return {f'x{i}': val for i, val in enumerate(state_shifted)}
        
#         def update_scaler(self, state):
#             """Инкрементальное обновление скейлера (только среднее)."""
#             state = np.array(state)
#             self.n_seen += 1
#             self.mean += (state - self.mean) / self.n_seen
        
#         def get_q_values(self, state):
#             state_dict = self._state_to_dict(state)
#             q_vals = []
#             for model in self.q_models:
#                 pred = model.predict_one(state_dict)
#                 q_vals.append(pred if pred is not None else 0.0)
#             return np.array(q_vals)
        
#         def get_epsilon(self):
#             return 0.25 + 0.75 * (self.temperature - self.final_temp) / (1.0 - self.final_temp)
        
#         def choose_action(self, state):
#             state = np.array(state).flatten()
            
#             if len(state) != self.env.J.shape[0]:
#                 return np.random.randint(0, self.env.N)
            
#             # Физическая эвристика
#             try:
#                 interaction = np.dot(self.env.J, state)
#                 local_contrib = -2 * state * interaction
#             except:
#                 local_contrib = np.zeros(self.N)
            
#             # Предсказание Q-значений
#             try:
#                 q_values = self.get_q_values(state)
#             except:
#                 q_values = np.zeros(self.N)
            
#             # Комбинируем с небольшим весом эвристики
#             combined = q_values + 0.15 * local_contrib
            
#             # ε-greedy
#             if self.rng.random() < self.get_epsilon():
#                 return self.rng.randint(0, self.N)
#             else:
#                 return int(np.argmax(combined))
        
#         def learn(self, episodes=1000, patience=None):
#             no_improve = 0
#             best_seen = self.env.best_energy
#             if patience is None:
#                 patience = 3 * self.N
#             for episode in range(episodes):
#                 self.temperature = max(self.final_temp, self.temperature * self.temp_decay_rate)
                
#                 state = self.env.get_state()
#                 action = self.choose_action(state)
#                 reward = self.env.flip_spin(action)
#                 next_state = self.env.get_state()
                
#                 # Обновляем скейлер
#                 self.update_scaler(state)
                
#                 # Вычисляем цель для Q-обучения
#                 next_q = self.get_q_values(next_state)
#                 target = reward + self.gamma * np.max(next_q)
                
#                 # Обучаем модель выбранного действия
#                 state_dict = self._state_to_dict(state)
#                 self.q_models[action].learn_one(state_dict, target)
                
#                 # Ранний останов, если нет улучшений
#                 if self.env.best_energy < best_seen:
#                     best_seen = self.env.best_energy
#                     no_improve = 0
#                 else:
#                     no_improve += 1
#                     if no_improve >= patience:
#                         break


# if __name__ == "__main__":
#     J_all = np.load("test_matrices.npy")[:30]
#     all_spins = []
#     all_energies = []
    
#     base_seed = GLOBAL_SEED
#     for idx, J in enumerate(tqdm(J_all, desc="Обработка графов")):
#         graph_seed = base_seed + idx
        
#         # Несколько перезапусков с разными инициализациями
#         restarts = 3
#         best_energy_run = np.inf
#         best_state_run = None
#         for r in range(restarts):
#             seed_r = graph_seed + 1000 * r
#             env = IsingEnvironment(J, seed=seed_r)
#             agent = TreeQAgent(env, gamma=0.95,
#                                initial_temp=1.0, final_temp=0.01, 
#                                temp_decay_rate=0.9995, seed=seed_r)
            
#             # Обучаем агента (меньше эпизодов, но с полировкой и ранним остановом)
#             agent.learn(episodes=3000, patience=2 * env.N)
#             env.greedy_polish(max_steps=2 * env.N)
            
#             if env.best_energy < best_energy_run:
#                 best_energy_run = env.best_energy
#                 best_state_run = env.get_best_state()
        
#         # Используем лучший результат среди перезапусков
#         all_spins.append(best_state_run)
#         all_energies.append(best_energy_run)
#         print(f"Граф {idx+1}: Лучшая энергия = {best_energy_run:.2f}")
    
#     mean_energy = np.mean(all_energies)
#     print(f"\nСредняя энергия: {mean_energy:.2f}")

Обработка графов:   3%|▎         | 1/30 [00:00<00:09,  3.06it/s]

Граф 1: Лучшая энергия = -208.00


Обработка графов:   7%|▋         | 2/30 [00:00<00:11,  2.38it/s]

Граф 2: Лучшая энергия = -192.00


Обработка графов:  10%|█         | 3/30 [00:01<00:11,  2.43it/s]

...

Граф 28: Лучшая энергия = -172.00


Обработка графов:  97%|█████████▋| 29/30 [00:12<00:00,  2.54it/s]

Граф 29: Лучшая энергия = -188.00


Обработка графов: 100%|██████████| 30/30 [00:13<00:00,  2.27it/s]

Граф 30: Лучшая энергия = -209.00

Средняя энергия: -193.07


In [3]:
# Simulated Annealing with tabu, reheating, and occasional 2-spin moves
import numpy as np

class SimulatedAnnealingAgent:
    def __init__(self, env, seed=None,
                 initial_temp=3.0, final_temp=0.02, alpha=0.997,
                 tabu_tenure=10, pair_move_prob=0.15,
                 top_k=None, reheating_factor=1.5, stagnation_patience=250):
        self.env = env
        self.rng = np.random.RandomState(seed)
        self.N = env.N
        self.temperature = initial_temp
        self.initial_temp = initial_temp
        self.final_temp = final_temp
        self.alpha = alpha
        self.tabu_tenure = tabu_tenure
        self.tabu = np.zeros(self.N, dtype=int)
        self.pair_move_prob = pair_move_prob
        self.top_k = self.N if top_k is None else min(top_k, self.N)
        self.reheating_factor = reheating_factor
        self.stagnation_patience = stagnation_patience
        self.steps_since_improve = 0
        self.best_energy_seen = env.best_energy
    
    def decay_temp(self):
        self.temperature = max(self.final_temp, self.temperature * self.alpha)
    
    def reheat_if_needed(self):
        if self.steps_since_improve >= self.stagnation_patience:
            self.temperature = min(self.initial_temp, self.temperature * self.reheating_factor)
            self.steps_since_improve = 0
    
    def compute_deltas(self):
        return np.array([self.env.delta_energy_if_flip(i) for i in range(self.N)], dtype=float)
    
    def two_flip_delta(self, i, j):
        s = self.env.state
        h_i = np.dot(self.env.J[i], s)
        h_j = np.dot(self.env.J[j], s)
        return 2*s[i]*h_i + 2*s[j]*h_j - 4*self.env.J[i, j]*s[i]*s[j]
    
    def step(self):
        deltas = self.compute_deltas()
        aspiration = (self.env.energy + deltas.min()) < self.env.best_energy
        allowed = self.tabu <= 0
        if not aspiration:
            candidates = np.where(allowed)[0]
        else:
            candidates = np.arange(self.N)
        if candidates.size == 0:
            candidates = np.arange(self.N)
        # focus top-k best deltas
        order = np.argsort(deltas[candidates])
        k = min(self.top_k, candidates.size)
        cand = candidates[order[:k]]
        # softmax over -ΔE / T
        scaled = -deltas[cand] / max(1e-9, self.temperature)
        scaled -= scaled.max()
        probs = np.exp(scaled)
        probs /= probs.sum()
        i = int(self.rng.choice(cand, p=probs))
        # maybe try 2-spin move if stuck or by probability
        do_pair = (self.rng.rand() < self.pair_move_prob) or (not aspiration and deltas.min() >= 0)
        if do_pair and self.N >= 2:
            subset = cand
            if subset.size < 2:
                subset = np.arange(self.N)
            best_pair = None
            best_pair_delta = np.inf
            for j in subset[:min(32, subset.size)]:
                if j == i:
                    continue
                if not aspiration and (self.tabu[j] > 0):
                    continue
                d2 = self.two_flip_delta(i, j)
                if d2 < best_pair_delta:
                    best_pair_delta = d2
                    best_pair = j
            if best_pair is not None:
                accept = (best_pair_delta < 0) or (self.rng.rand() < np.exp(-best_pair_delta / max(1e-9, self.temperature)))
                if accept:
                    self.env.flip_spin(i)
                    self.env.flip_spin(best_pair)
                    self.tabu[i] = self.tabu_tenure
                    self.tabu[best_pair] = self.tabu_tenure
                    improved = self.env.energy < self.best_energy_seen - 1e-12
                    self.best_energy_seen = min(self.best_energy_seen, self.env.energy)
                    self.steps_since_improve = 0 if improved else self.steps_since_improve + 1
                    self.tabu -= 1
                    self.decay_temp()
                    self.reheat_if_needed()
                    return
        # single flip Metropolis
        d = deltas[i]
        accept = (d < 0) or (self.rng.rand() < np.exp(-d / max(1e-9, self.temperature)))
        if accept:
            self.env.flip_spin(i)
            self.tabu[i] = self.tabu_tenure
            improved = self.env.energy < self.best_energy_seen - 1e-12
            self.best_energy_seen = min(self.best_energy_seen, self.env.energy)
            self.steps_since_improve = 0 if improved else self.steps_since_improve + 1
        else:
            self.steps_since_improve += 1
        self.tabu -= 1
        self.decay_temp()
        self.reheat_if_needed()
    
    def run(self, steps):
        for _ in range(steps):
            self.step()
    
    def greedy_polish(self, max_steps=500):
        for _ in range(max_steps):
            deltas = self.compute_deltas()
            i = int(np.argmin(deltas))
            if deltas[i] < -1e-12:
                self.env.flip_spin(i)
            else:
                break

# Run SA on all graphs
if __name__ == "__main__":
    import numpy as np
    from tqdm import tqdm

    J_all = np.load("test_matrices.npy")
    sa_spins = []
    sa_energies = []
    base_seed = 42

    for idx, J in enumerate(tqdm(J_all, desc="SA: Обработка графов")):
        graph_seed = base_seed + idx
        env = IsingEnvironment(J, seed=graph_seed)
        best_energy_run = np.inf
        best_state_run = None

        num_restarts = 6
        steps_per_restart = max(3000, 60 * env.N)
        for r in range(num_restarts):
            # random restart
            env.state = env.rng.choice([-1, 1], size=env.N)
            env.energy = env.calculate_energy(env.state)
            env.no_improvement_streak = 0
            env.best_state = env.state.copy()
            env.best_energy = env.energy

            sa = SimulatedAnnealingAgent(
                env,
                seed=graph_seed + r,
                initial_temp=3.0,
                final_temp=0.02,
                alpha=0.997,
                tabu_tenure=10,
                pair_move_prob=0.2,
                top_k=max(8, env.N // 4),
                reheating_factor=1.6,
                stagnation_patience=max(200, 5 * env.N)
            )

            sa.run(steps_per_restart)
            sa.greedy_polish(max_steps=3 * env.N)

            if env.best_energy < best_energy_run:
                best_energy_run = env.best_energy
                best_state_run = env.get_best_state()

        sa_spins.append(best_state_run)
        sa_energies.append(best_energy_run)
        print(f"SA Граф {idx+1}: Лучшая энергия = {best_energy_run:.6f}")

    sa_mean_energy = float(np.mean(sa_energies))
    print(f"\nSA Средняя энергия: {sa_mean_energy:.6f}")


SA: Обработка графов:   0%|          | 1/200 [00:02<08:55,  2.69s/it]

SA Граф 1: Лучшая энергия = -216.000000


SA: Обработка графов:   1%|          | 2/200 [00:05<08:54,  2.70s/it]

SA Граф 2: Лучшая энергия = -208.000000


SA: Обработка графов:   2%|▏         | 3/200 [00:08<08:53,  2.71s/it]

...

SA Граф 198: Лучшая энергия = -196.000000


SA: Обработка графов: 100%|█████████▉| 199/200 [09:20<00:02,  2.72s/it]

SA Граф 199: Лучшая энергия = -209.000000


SA: Обработка графов: 100%|██████████| 200/200 [09:23<00:00,  2.82s/it]

SA Граф 200: Лучшая энергия = -207.000000

SA Средняя энергия: -205.255000


In [ ]:
header=','.join(str(i) for i in range(1, 51))
pd.DataFrame(sa_spins).to_csv("spins.csv", index=False, header=header)
pd.DataFrame(sa_energies, columns=["Energy"]).to_csv("energies.csv", index=False)